In [2]:
%run imports.py

# Preprocessing

In [105]:
# Read in the food nutrients and recommended daily allowance DataFrames.
nutrients_df = pd.read_msgpack('nutrients-nl.msg')
rda_df = pd.read_msgpack('rda.msg')

In [106]:
# Convert to numeric.
nutrients_df = nutrients_df \
    .replace(r'([0-9.]+)[^0-9]+', r'\1', regex=True) \
    .replace('--', np.nan) \
    .replace('spoor', 0)
nutrients_df.iloc[:, 3:] = nutrients_df.iloc[:, 3:].astype(np.float64)

In [107]:
# Convert all measurements to grams.
def to_grams(df):
    def col_to_grams(col):
        si = re.findall(r'\(([^A]?)(?:g|J|L|cal|IU|%)', col.name)
        if len(si) != 1:
            raise ValueError(f'Bad column format: {si} ({col.name})')
        si = si[0]
        if not si:
            return col
        elif si == 'k':
            return 1e3 * col
        elif si == 'm':
            return 1e-3 * col
        elif si in ['µ', 'μ', 'u']:  # These two mus are different chars.
            return 1e-6 * col
        else:
            raise ValueError(f'Unknown unit: {si} ({col.name})')
    for col in df.select_dtypes(exclude=['object']).columns:
        df[col] = col_to_grams(df[col])
    return df

nutrients_df = to_grams(nutrients_df)
rda_df = to_grams(rda_df)

In [108]:
# Remove RDA columns that are not measured in the food nutrients DataFrame.
verified_missing = [
    # Elements
    'Chromium(μg/d)',
    'Fluoride(mg/d)',
    'Manganese(mg/d)',
    'Molybdenum(μg/d)',
    'Chloride(g/d)',
    # Vitamins
    'Pantothenic Acid(mg/d)',
    'Biotin(μg/d)',
    'Choline(mg/d)g',
    # Macronutrients
    'Total Watera(L/d)'
]
rda_df = rda_df.drop(verified_missing, axis=1)

In [109]:
# Rename RDA columns to match the food nutrients column names.
exceptions = {
    # Elements
    'Copper(μg/d)': 'Koper (mg)',
    'Iodine(μg/d)': 'Jodium (ug)',
    'Iron(mg/d)': 'IJzer totaal (mg)',
    'Phosphorus(mg/d)': 'Fosfor (mg)',
    'Zinc(mg/d)': 'Zink (mg)',
    'Potassium(g/d)': 'Kalium (mg)',
    'Sodium(g/d)': 'Natrium (mg)',
    # Vitamins
    'Vitamin A(μg/d)a': 'Retinol activiteit equiv.(RAE) (ug)',
    'Vitamin C(mg/d)': 'Vitamine C (mg)',
    'Vitamin D(μg/d)b,c': 'Cholecalciferol - vit D3 (ug)',  # 'Vitamine D totaal (ug)'
    'Vitamin E(mg/d)d': 'Alfa-tocoferol (mg)',  # 'Vitamine E totaal (mg)',
    'Vitamin K(μg/d)': 'Vitamine K totaal (ug)',
    'Thiamin(mg/d)': 'Vitamine B1 (mg)',
    'Riboflavin(mg/d)': 'Vitamine B2 (mg)',
    'Niacin(mg/d)e': 'Nicotinezuur (mg)',
    'Vitamin B6(mg/d)': 'Vitamine B6 (mg)',
    'Folate(μg/d)f': 'Foliumzuur (ug)',
    'Vitamin B12(μg/d)': 'Vitamine B12 (ug)',
    # Macronutrients
    'Carbohydrate(g/d)': 'Koolhydraten totaal (g)',
    'Total Fiber(g/d)': 'Voedingsvezel totaal (g)',
    'Fat(g/d)': 'Vet totaal (g)',
    'Linoleic Acid(g/d)': 'C18:2 n-6 cis (linolzuur) (g)',
    'α-Linolenic Acid(g/d)': 'C18:3 n-3 cis (ALA) (g)',
    'Proteinb(g/d)': 'Eiwit totaal (g)'
}
columns = {**exceptions}
for col in rda_df.columns:
    if col in exceptions:
        continue
    query = re.findall(r'^([a-z\s]+)[^a-z]', col.lower())[0]
    results = [c for c in nutrients_df.columns if query in c.lower()]
    if len(results) != 1:
        raise ValueError(f'Found {results} for "{query}" in column {col}')
    columns[col] = results[0]
rda_df = rda_df.rename(columns=columns)
assert all([col in nutrients_df.columns for col in rda_df.columns])

# MILP model that optimizes health, time, cost

Possible losses:
- Time (Preparation): Waste
- Time (Shopping): Number of items
- Time (Eating): Meal weight
- Health: Recommended Dietary Allowances (Adequate Intake)
- Money: Cost

In [234]:
# Extract the RDA for the user.
me = 'Males: 19–30 y'
my_rda = rda_df.loc[me].dropna()
# Drop Magnesium because it is very difficult to find in foods.
my_rda.drop(['Magnesium (mg)', 'Foliumzuur (ug)', 'Selenium (ug)'], axis=0, inplace=True) 
# Keep only the nutrients we will be evaluating on and
# normalize the nutrients to (% of RDA) / (household measure).
pct_rda_df = nutrients_df.fillna(0.)[['Voedselgroep', 'Voedingsmiddel'] + my_rda.index.tolist()].copy()
pct_rda_df.loc[:, my_rda.index] = pct_rda_df.loc[:, my_rda.index].divide(my_rda, axis=1)
pct_rda_df.sample(5)

,Voedselgroep,Voedingsmiddel,Calcium (mg),Koper (mg),Jodium (ug),IJzer totaal (mg),Fosfor (mg),Zink (mg),Kalium (mg),Natrium (mg),Retinol activiteit equiv.(RAE) (ug),Vitamine C (mg),Cholecalciferol - vit D3 (ug),Alfa-tocoferol (mg),Vitamine K totaal (ug),Vitamine B1 (mg),Vitamine B2 (mg),Nicotinezuur (mg),Vitamine B6 (mg),Vitamine B12 (ug),Koolhydraten totaal (g),Voedingsvezel totaal (g),C18:2 n-6 cis (linolzuur) (g),C18:3 n-3 cis (ALA) (g),Eiwit totaal (g)
NEVO-code,,,,,,,,,,,,,,,,,,,,,,,,,
2783,Brood,Brood meergranen- m div zaden obv bloem,0.046,26.666667,4.706667,2.1250,0.225714,10.727273,0.040851,0.296000,0.000000,0.000000,0.000000,0.066667,0.000,9.166667,5.384615,1.0000,72.307692,0.000000,0.310769,1.421053,1.058824,2.500,0.223214
2423,"Vetten, oliën en hartige sauzen",Halvarineproduct Blue Band Idee,0.600,0.000000,0.100000,0.0125,0.000000,0.000000,0.000000,0.088000,0.888889,0.000000,0.000000,0.000000,0.000,1.416667,0.000000,0.0000,1.615385,0.158333,0.011538,0.000000,0.705882,19.375,0.053571
3270,Preparaten,Duocal ss poeder Milupa,0.005,0.000000,0.000000,0.0000,0.007143,0.000000,0.001064,0.013333,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,5.592308,0.000000,0.000000,0.000,0.000000
305,Melk en melkproducten,Kwark magere,0.128,2.222222,0.993333,0.0000,0.142857,5.181818,0.034681,0.030000,0.006667,0.011111,0.000000,0.000000,2.125,3.333333,2.307692,0.0625,0.769231,0.291667,0.037692,0.000000,0.000000,0.000,0.151786
1255,Vis,Makreel in water blik,0.020,31.111111,0.326667,0.2500,0.128571,2.454545,0.048511,0.277333,0.041111,0.000000,2.866667,0.000000,0.000,2.500000,20.769231,3.3125,10.769231,28.916667,0.000000,0.000000,0.176471,0.175,0.332143


In [235]:
# Drop exotic or difficult to find foods.
drop = [
    #3131,  # Graanproducten en bindmiddelen	Ontbijtproduct Pyjamapapje fijne tarwe granen,
    439,   # Graanproducten en bindmiddelen	Gelatine
    83,    # Eieren	Ei kippen- rauw gem
    #1233,  # Kruiden en specerijen	Sambal oelek Na-
    #1232,  # Sambal oelek normaal
]
pct_rda_df.drop(drop, axis=0, inplace=True)
is_preparaat = \
    pct_rda_df.Voedingsmiddel.str.contains('iroop') | \
    pct_rda_df.Voedingsmiddel.str.contains('nergy') | \
    pct_rda_df.Voedingsmiddel.str.contains('ortdrank') | \
    pct_rda_df.Voedingsmiddel.str.contains('oeder') | \
    pct_rda_df.Voedingsmiddel.str.contains('iksap') | \
    pct_rda_df.Voedingsmiddel.str.contains('portvoed') | \
    pct_rda_df.Voedingsmiddel.str.contains('hicken') | \
    pct_rda_df.Voedingsmiddel.str.contains('owder')
pct_rda_df = pct_rda_df.loc[~is_preparaat, :]

In [238]:
# Variables.
purchases = pulp.LpVariable.dicts(
    'purchases',
    indexs=range(len(pct_rda_df)),
    lowBound=0,
    upBound=1,
    cat='Binary')
portions = pulp.LpVariable.dicts(
    'portions',
    indexs=range(len(pct_rda_df)),
    lowBound=0,
    upBound=50,
    cat='Continuous')
rda_errors = pulp.LpVariable.dicts(
    'rda_errors',
    indexs=range(len(my_rda)),
    lowBound=0,
    upBound=2,
    cat='Continuous')
max_rda_error = pulp.LpVariable(
    'max_rda_error',
    lowBound=0,
    upBound=2,
    cat='Continuous')
total_weight = pulp.LpVariable(
    'total_weight',
    lowBound=0,
    upBound=2000,
    cat='Continuous')

# Cost function.
rda_vs_weight = 0.95
max_foods = 14
max_weight_per_food = 200
max_spices_weight = 3
prob = pulp.LpProblem('food', pulp.LpMinimize)
# prob += rda_vs_weight * max_rda_error + (1 - rda_vs_weight) * total_weight / 1e3
prob += rda_vs_weight * pulp.lpSum(rda_errors) / len(rda_errors) + 0.25 * max_rda_error + (1 - rda_vs_weight) * total_weight / 1e3

# Constraints:
# - Total weight.
prob += total_weight == pulp.lpSum(100 * portions[portion] for portion in portions)
# - Max RDA error.
for rda_error in rda_errors:
    prob += max_rda_error >= rda_errors[rda_error]
# - Not too much spices.
for portion in portions:
    if pct_rda_df.iat[portion, 0] == 'Kruiden en specerijen':
        prob += 100 * portions[portion] <= max_spices_weight
# - No more than max_weight per food.
for portion in portions:
    prob += 100 * portions[portion] <= max_weight_per_food
# - No more than max_foods.
prob += pulp.lpSum(purchases[purchase] for purchase in purchases) <= max_foods
# - Purchase is binary.
for portion, purchase in zip(portions, purchases):
    prob += portions[portion] <= 10000 * purchases[purchase]
# - RDA error is absolute error per nutrient.
for rda_error in rda_errors:
    raw_error = pulp.lpSum(pct_rda_df.iat[portion, rda_error + 2] * portions[portion] for portion in portions) - 1.
    prob += -rda_errors[rda_error] <= raw_error
    prob += raw_error <= rda_errors[rda_error]
    # prob += raw_error == rda_errors[rda_error]
prob.writeLP('food.lp')

In [239]:
solver = pulp.solvers.COIN_CMD(
    maxSeconds=30,
    options=['feasibilityPump off', 'probing off', 'knapsack off', 'clique off', 'flowcover off'])
prob.solve(solver=solver)

print(f'Med RDA error is {np.median([rda_errors[i].value() for i in rda_errors])}')
print(f'Avg RDA error is {np.mean([rda_errors[i].value() for i in rda_errors])}')
print(f'Max RDA error is {np.max([rda_errors[i].value() for i in rda_errors])}')
print(f'Total weight is {total_weight.value()}g')

x = np.array([portions[p].value() for p in portions])
tmp = pct_rda_df.iloc[:, :2].loc[x > 0].copy()
tmp['Gewicht'] = 100 * x[x > 0]
tmp
pct_rda_df.iloc[:, 2:].loc[x > 0].multiply(x[x > 0], axis=0)
pct_rda_df.iloc[:, 2:].loc[x > 0].multiply(x[x > 0], axis=0).sum(axis=0)

0

Med RDA error is 0.049559443
Avg RDA error is 0.12606019669565213
Max RDA error is 0.33224947
Total weight is 537.14015g


,Voedselgroep,Voedingsmiddel,Gewicht
NEVO-code,,,
85,Eieren,Eidooier kippen- rauw,29.692627
126,Groenten,Sterkers rauw,2.175914
317,"Vetten, oliën en hartige sauzen",Olie zonnebloem-,2.056726
1140,Groenten,Groente erwtensoep- diepvries onbereid,200.000000
1212,Diversen,Zout mineraal- Na- LoSalt,7.824516
1214,Sojaproducten en vegetarische producten,Ketjap zoet Na-,1.519350
1612,Vis,Forel regenboog- bereid in magnetron z toev,2.214068
1897,Diversen,Zeewier kelp rauw,1.313881
2005,Graanproducten en bindmiddelen,Ontbijtproduct Special K Original Kellogg's,21.304192


,Calcium (mg),Koper (mg),Jodium (ug),IJzer totaal (mg),Fosfor (mg),Zink (mg),Kalium (mg),Natrium (mg),Retinol activiteit equiv.(RAE) (ug),Vitamine C (mg),Cholecalciferol - vit D3 (ug),Alfa-tocoferol (mg),Vitamine K totaal (ug),Vitamine B1 (mg),Vitamine B2 (mg),Nicotinezuur (mg),Vitamine B6 (mg),Vitamine B12 (ug),Koolhydraten totaal (g),Voedingsvezel totaal (g),C18:2 n-6 cis (linolzuur) (g),C18:3 n-3 cis (ALA) (g),Eiwit totaal (g)
NEVO-code,,,,,,,,,,,,,,,,,,,,,,,
85,0.044539,0.049488,0.136982,0.218983,0.262568,0.116071,0.006444,0.007918,0.177166,0.000000,0.890779,0.000000,0.000000,0.049488,0.114202,0.000000,0.036545,0.047013,0.000457,0.000000,0.036679,0.005567,0.088548
126,0.005440,0.217591,0.001306,0.006800,0.002487,0.090993,0.002315,0.001160,0.000000,0.012088,0.000000,0.000000,0.982606,0.018133,0.016738,0.001360,0.284543,0.000000,0.000000,0.001890,0.000000,0.000000,0.000777
317,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.667751,0.009769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000032,0.000000,0.751310,0.012855,0.001102
1140,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.033333,0.335556,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.178947,0.000000,0.000000,0.071429
1212,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.576018,0.683341,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1214,0.000912,0.000000,0.000000,0.100657,0.001085,0.000000,0.020366,0.000810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006428,0.000400,0.000000,0.000000,0.001221
1612,0.000443,0.000000,0.003690,0.002768,0.006958,0.000000,0.001578,0.001299,0.000344,0.000000,0.000000,0.000000,0.000000,0.313660,0.051094,0.156369,0.068125,0.004613,0.000000,0.000000,0.001302,0.000138,0.009370
1897,0.002207,0.189783,0.661583,0.047628,0.000788,0.146916,0.000249,0.002041,0.000000,0.000438,0.000000,0.000000,0.000000,0.054745,0.151602,0.004106,0.020214,0.000000,0.000839,0.004495,0.000000,0.000000,0.000399
2005,0.012783,0.000000,0.034087,0.308911,0.051739,0.290512,0.009972,0.063913,0.000000,0.317196,0.000000,0.000000,0.003551,0.319563,0.376920,0.400785,0.376920,0.037282,0.129464,0.025229,0.075191,0.000000,0.034239


Calcium (mg)                           1.000000
Koper (mg)                             1.000000
Jodium (ug)                            1.000000
IJzer totaal (mg)                      1.000000
Fosfor (mg)                            1.110496
Zink (mg)                              1.000000
Kalium (mg)                            0.667751
Natrium (mg)                           1.332249
Retinol activiteit equiv.(RAE) (ug)    0.845194
Vitamine C (mg)                        0.769200
Cholecalciferol - vit D3 (ug)          1.000000
Alfa-tocoferol (mg)                    0.667751
Vitamine K totaal (ug)                 1.000000
Vitamine B1 (mg)                       1.000000
Vitamine B2 (mg)                       0.973769
Nicotinezuur (mg)                      0.807032
Vitamine B6 (mg)                       1.049559
Vitamine B12 (ug)                      0.717125
Koolhydraten totaal (g)                1.000000
Voedingsvezel totaal (g)               1.000000
C18:2 n-6 cis (linolzuur) (g)          1